In [ ]:
import sys
sys.path.append('/home/rrg0013@auburn.edu/git/nodules/src/')
sys.path.append('/home/rrg0013@auburn.edu/git/nodules/models/')
sys.path.append('/home/rrg0013@auburn.edu/git/nodules/keras/')

In [ ]:
import fpr_xrv2_1c3 as m
import generatorModel as gen
import kerasUtil as ku
from sklearn.metrics import roc_curve, f1_score, auc
from glob import glob

In [ ]:
import os
import scipy.misc
import numpy as np
import matplotlib.pyplot as plt

### Enter paths

In [ ]:
PATH = '/scr/data/nodules/testseg/cands/train2/seven/'
WEIGHT = '/scr/data/nodules/init3dConv/073117/7003/64-no7003-accuracy-073117.h5'

In [ ]:
def loadpng1(cfg, full):
  path = cfg
  images, labels = [], []
  for parent, subdir, files in os.walk(path):
    for file in files:
      image = scipy.misc.imread(os.path.join(parent,file))
      image = ku.formatImage(image,full)
      images.append(np.array(image))
      label = file
      if label[1] == '.':
        label = label[2:]
      elif label[2] == '.':
        label = label[3:]
      label = str(label)
      meta = label.split('_')
      label = {'labels':meta[0]}
      labels.append(label)
  if path[-4:] == '.png':
    image = scipy.misc.imread(path)
    image = ku.formatImage(image,full)
    images.append(np.array(image))
    label = path.split('/')
    label = label[-1]
    if label[1] == '.':
      label = label[2:]
    elif label[2] == '.':
      label = label[3:]
    label = str(label)
    meta = label.split('_')
    label = {'labels':meta[0]}
    labels.append(label)
  return images, labels

In [ ]:
images, labels = loadpng1(PATH, True)
images, labels = ku.cleanData(images, labels, 2)
images = ku.to3d(images, 2, True)
labels = ku.get2Labels(labels)
images = ku.convertArray(images)

In [ ]:
len(images)

In [ ]:
print len(images)

In [ ]:
mean=0.188313096763
std=0.1979406029

images -= mean
images /= std

In [ ]:
model = m.getModel(2,sz=32,full=True)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [ ]:
model.load_weights(WEIGHT)

In [ ]:
preds = model.predict(images, batch_size=64)

In [ ]:
fpr, tpr, _ = roc_curve(labels[:,0],preds[:,0])
area = auc(fpr, tpr)
print area
plt.plot(fpr,tpr)
plt.savefig('/home/rrg0013@auburn.edu/four_roc.png')  
plt.clf()

### Found of all candidate nodules 

In [ ]:
tmp = np.zeros(preds.shape[0])
tmp[preds[:,0] > preds[:,1]] = 1
pos = np.sum(tmp)
print pos

### False positives per scan 

In [ ]:
perscan = pos/89. 
print perscan

### Sensitivity 

In [ ]:
tlbl = np.zeros(preds.shape[0])
tlbl[labels[:,0]==1] = 1
tp_fn = np.sum(tlbl)
print tp_fn

In [ ]:
tmp2 = np.zeros(preds.shape[0])
tmp2[preds[:,0] > preds[:,1]] = 1
tmp2[tlbl==1] += 1
tmp3 = np.where(tmp2==2,1,0)
tp = np.sum(tmp3)
print tp

#### SENSITIVITY VALUE:

In [ ]:
sensitivity = tp/tp_fn
print sensitivity

### Specificity 

In [ ]:
tlbl = np.zeros(preds.shape[0])
tlbl[labels[:,1]==1] = 1
tn_fp = np.sum(tlbl)
print tn_fp

In [ ]:
tmp = np.zeros(preds.shape[0])
tmp[preds[:,1] > preds[:,0]] = 1
tn = np.sum(tmp)
print tn

#### SPECIFICITY VALUE:

In [ ]:
specificity = tn/tn_fp
print specificity